In [2]:
!pip install --quiet git+https://github.com/GeoscienceAustralia/AusPIX_DGGS.git

In [3]:
!pip install --quiet geojson shapely pygeoj matplotlib

In [4]:
import geojson
fname = "ACT_SA1_Black_Mountain.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

In [5]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -35.282
y_coord = 149.09

map1 = ipy.Map(center=[x_coord, y_coord], zoom=13)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'blue', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.3})
map1.add_layer(geo_json1)
map1

Map(center=[-35.282, 149.09], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [6]:
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from geojson import Feature, FeatureCollection, Polygon
import matplotlib, json

DGGS_RESOLUTION=10

list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION, True)  

list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon([bbox_coords])
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)
print(json.dumps(feature_collection, indent=2))

NameError: name 'get_cells_in_geojson' is not defined

In [5]:
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon
from geojson.utils import coords
from geojson import Feature, FeatureCollection
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS

def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     return ret
    
def geojson_to_shape(g):
    return shape(g)
    
def get_cells_in_geojson(geojson, resolution):
    list_cells = []
    for fea in geojson['features']:  # for feature in attribute table
        res_cells = get_cells_in_feature(fea, resolution)
        list_cells = list(list_cells + res_cells)
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = res_cells
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]

    return cells

DGGS_RESOLUTION=10
list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION)    
list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

#uncomment if you want to get a file output
#geojson_dump = geojson.dumps(feature_collection, indent=4, sort_keys=True)
#with open('poly.geojson', 'w') as json_file:
#    data = geojson.dump(feature_collection, json_file, indent=4, sort_keys=True)        

auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix
auspixengine.pj_rhealpix

num cells in bb =  812
total vertex in this poly 515
new poly 149.087858


TypeError: 'float' object is not iterable

In [ ]:
x_coord = -35.282
y_coord = 149.09

map2 = ipy.Map(center=[x_coord, y_coord], zoom=13)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'red', 
                          'fillOpacity': 0.3})
map2.add_layer(geo_json1)
map2.add_layer(geo_json2)

map2


In [ ]:
import requests
import json
#iterate over features in feature collection and query Loc-I API for features
def find_features_at_dggs_cell(dggs_cell_id):
    payload = {
        'dggs_cell' : dggs_cell_id
    }
    r = requests.get('https://api.loci.cat/api/v1/location/find-at-DGGS-cell', params=payload)
    json_res = r.json()
    return json_res


In [8]:
target_feature_type = 'asgs16_lga'

set_of_lga_features = set([])
#print(feature_collection)
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_lga_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))

    


Processing R7852344875
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852344883
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852344877
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852344878
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852344886
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852344887
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852344888
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347211
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347212
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347220
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing

Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348055
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348133
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347247
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347248
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347256
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347257
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347258
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348036
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348037
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348038
Adding htt

Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348305
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348313
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348314
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348315
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348323
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348324
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348325
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348403
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852348404
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347517
Adding htt

Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347801
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347802
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347724
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347725
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347803
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347804
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347805
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347726
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347727
Adding http://linked.data.gov.au/dataset/asgs2016/localgovernmentarea/89399
Processing R7852347728
Adding htt

In [39]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    print(r.url)
    print(r.text)
    res = r.json()
    #get the first geom result
    geojson_data = res['geometry'][0]
    print(geojson_data)
    return geojson_data



In [43]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=10)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


In [44]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data

for feature_uri in list(set_of_lga_features):
    geojson_data = get_geom(feature_uri)
    curr_geojson_layer = GeoJSON(data=geojson_data, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'yellow', 
                          'fillOpacity': 0.3})
    map3.add_layer(curr_geojson_layer)
map3.add_layer(geo_json1)
map3.add_layer(geo_json2)


In [45]:
map3

Map(center=[-35.282, 149.09], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…